In [3]:
import os

# Chemin du dossier contenant les fichiers
dossier = "./xTB_OPT_XYZ"

# Parcourir tous les fichiers du dossier
for fichier in os.listdir(dossier):
    if fichier.startswith("gdb ") and fichier.endswith(".xyz"):  # Vérifie si le nom commence par "gdb "
        nouveau_nom = fichier.replace("gdb ", "gdb")  # Supprime l'espace après "gdb"
        ancien_chemin = os.path.join(dossier, fichier)
        nouveau_chemin = os.path.join(dossier, nouveau_nom)
        os.rename(ancien_chemin, nouveau_chemin)
        print(f"Renommé : {fichier} → {nouveau_nom}")

print("Tous les fichiers ont été renommés ! ✅")


Renommé : gdb 64068_xtb1_opt.xyz → gdb64068_xtb1_opt.xyz
Renommé : gdb 30762_xtb1_opt.xyz → gdb30762_xtb1_opt.xyz
Renommé : gdb 88935_xtb1_opt.xyz → gdb88935_xtb1_opt.xyz
Renommé : gdb 23724_xtb1_opt.xyz → gdb23724_xtb1_opt.xyz
Renommé : gdb 127485_xtb1_opt.xyz → gdb127485_xtb1_opt.xyz
Renommé : gdb 115131_xtb1_opt.xyz → gdb115131_xtb1_opt.xyz
Renommé : gdb 30457_xtb1_opt.xyz → gdb30457_xtb1_opt.xyz
Renommé : gdb 24550_xtb1_opt.xyz → gdb24550_xtb1_opt.xyz
Renommé : gdb 133848_xtb1_opt.xyz → gdb133848_xtb1_opt.xyz
Renommé : gdb 49119_xtb1_opt.xyz → gdb49119_xtb1_opt.xyz
Renommé : gdb 127844_xtb1_opt.xyz → gdb127844_xtb1_opt.xyz
Renommé : gdb 130885_xtb1_opt.xyz → gdb130885_xtb1_opt.xyz
Renommé : gdb 96613_xtb1_opt.xyz → gdb96613_xtb1_opt.xyz
Renommé : gdb 10016_xtb1_opt.xyz → gdb10016_xtb1_opt.xyz
Renommé : gdb 109422_xtb1_opt.xyz → gdb109422_xtb1_opt.xyz
Renommé : gdb 26915_xtb1_opt.xyz → gdb26915_xtb1_opt.xyz
Renommé : gdb 22213_xtb1_opt.xyz → gdb22213_xtb1_opt.xyz
Renommé : gdb 74202

In [4]:
import os
import subprocess

import csv
import pickle
from pathlib import Path

# Chemins vers les outils
orca_path = "orca"  # Remplacez par le chemin vers ORCA

# Dossier contenant les fichiers .xyz générés par xTB
xtb_dir = "./xTB_OPT_XYZ"
orca_dir = "path/orca_results"
os.makedirs(orca_dir, exist_ok=True)

# Fonction pour vérifier et formater les fichiers XYZ
def format_xyz_file(input_file, output_file):
    with open(input_file, "r") as f:
        lines = f.readlines()
    
    # Vérifier si la première ligne est un nombre (nombre d'atomes)
    if not lines[0].strip().isdigit():
        # Ajouter le nombre d'atomes et une ligne vide
        num_atoms = len(lines)
        formatted_lines = [f" {num_atoms}\n", "\n"] + lines
    else:
        formatted_lines = lines
    
    # Écrire le fichier formaté
    with open(output_file, "w") as f:
        f.writelines(formatted_lines)

# Fonction pour supprimer les fichiers inutiles, sauf ceux avec certaines extensions
def clean_up_files(directory, keep_extensions=(".xyz", ".txt", ".out", ".inp")):
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if not file.endswith(keep_extensions):
            os.remove(file_path)
            print(f"Supprimé : {file_path}")

# Parcourir tous les fichiers XYZ
for file in os.listdir(xtb_dir):
    if file.endswith(".xyz"):
        file_path = os.path.join(xtb_dir, file)
        mol_id = os.path.splitext(file)[0]

        # Formater le fichier XYZ
        formatted_xyz_file = os.path.join(orca_dir, f"{mol_id}_formatted.xyz")
        format_xyz_file(file_path, formatted_xyz_file)

        # Fichiers d'entrée et de sortie ORCA
        orca_input_file = os.path.join(orca_dir, f"Test_{mol_id}.inp")  # Modèle Test.inp
        orca_output_file = os.path.join(orca_dir, f"Test_{mol_id}.out")  # Modèle Test.out

        # Générer le fichier d'entrée ORCA
        with open(orca_input_file, "w") as f:
            f.write(f"""! B3LYP 6-31G** TightSCF
* xyzfile 0 1 {formatted_xyz_file}
""")

        # Exécuter ORCA et capturer les erreurs
        try:
            # Rediriger explicitement les flux vers le fichier de sortie
            with open(orca_output_file, "w") as orca_out:
                result = subprocess.run(
                    [orca_path, orca_input_file],  # Commande ORCA
                    stdout=orca_out,               # Rediriger stdout vers le fichier .out
                    stderr=subprocess.PIPE,        # Capturer stderr pour vérifier les erreurs
                    text=True,
                    check=True,
                )
            print(f"ORCA terminé avec succès pour {mol_id}")
        except subprocess.CalledProcessError as e:
            # Afficher les messages d'erreur d'ORCA
            print(f"ERREUR : ORCA a échoué pour {mol_id}")
            print(f"Code de sortie : {e.returncode}")
            print(f"Erreur standard (stderr) : {e.stderr}")

            # Enregistrer les erreurs dans un fichier de log
            error_log_file = os.path.join(orca_dir, f"{mol_id}_error.log")
            with open(error_log_file, "w") as log:
                log.write(f"Code de sortie : {e.returncode}\n")
                log.write("Erreur standard (stderr) :\n")
                log.write(e.stderr + "\n")

        # Supprimer les fichiers inutiles
        clean_up_files(orca_dir)

        # Supprimer le fichier XYZ formaté temporaire
        if os.path.exists(formatted_xyz_file):
            os.remove(formatted_xyz_file)
            print(f"Supprimé le fichier temporaire : {formatted_xyz_file}")

ORCA terminé avec succès pour gdb28851_xtb1_opt
Supprimé : path/orca_results/Test_gdb28851_xtb1_opt.densitiesinfo
Supprimé : path/orca_results/Test_gdb28851_xtb1_opt.gbw
Supprimé : path/orca_results/Test_gdb28851_xtb1_opt.densities
Supprimé : path/orca_results/Test_gdb28851_xtb1_opt.bibtex
Supprimé le fichier temporaire : path/orca_results/gdb28851_xtb1_opt_formatted.xyz
ORCA terminé avec succès pour gdb25197_xtb1_opt
Supprimé : path/orca_results/Test_gdb25197_xtb1_opt.densitiesinfo
Supprimé : path/orca_results/Test_gdb25197_xtb1_opt.gbw
Supprimé : path/orca_results/Test_gdb25197_xtb1_opt.bibtex
Supprimé : path/orca_results/Test_gdb25197_xtb1_opt.densities
Supprimé le fichier temporaire : path/orca_results/gdb25197_xtb1_opt_formatted.xyz
ORCA terminé avec succès pour gdb37498_xtb1_opt
Supprimé : path/orca_results/Test_gdb37498_xtb1_opt.gbw
Supprimé : path/orca_results/Test_gdb37498_xtb1_opt.densities
Supprimé : path/orca_results/Test_gdb37498_xtb1_opt.densitiesinfo
Supprimé : path/orca

In [ ]:
import os
import subprocess
import csv
import pickle
from pathlib import Path
import tempfile

# Chemins vers les outils
orca_path = "orca"  # Remplacez par le chemin vers ORCA

# Dossier contenant les datasets de fichiers .xyz générés par xTB
input_dirs = ["./Dataset1", "./Dataset2"]  # Ajouter plusieurs datasets ici
output_base_dir = Path("./orca_results")
output_base_dir.mkdir(exist_ok=True)

# Fonction pour formater les fichiers XYZ
def format_xyz_file(input_file, output_file):
    with open(input_file, "r") as f:
        lines = f.readlines()
    
    if not lines[0].strip().isdigit():
        num_atoms = len(lines)
        formatted_lines = [f" {num_atoms}\n", "\n"] + lines
    else:
        formatted_lines = lines
    
    with open(output_file, "w") as f:
        f.writelines(formatted_lines)

# Fonction pour exécuter ORCA
def run_orca(mol_id, formatted_xyz_file, output_dir):
    orca_input_file = output_dir / f"Test_{mol_id}.inp"
    orca_output_file = output_dir / f"Test_{mol_id}.out"

    with open(orca_input_file, "w") as f:
        f.write(f"""! B3LYP 6-31G** TightSCF
* xyzfile 0 1 {formatted_xyz_file}
""")

    try:
        with open(orca_output_file, "w") as orca_out:
            subprocess.run(
                [orca_path, str(orca_input_file)],
                stdout=orca_out,
                stderr=subprocess.PIPE,
                text=True,
                check=True,
            )
        print(f"✅ ORCA terminé avec succès pour {mol_id}")
    except subprocess.CalledProcessError as e:
        print(f"❌ ERREUR : ORCA a échoué pour {mol_id}")
        error_log_file = output_dir / f"{mol_id}_error.log"
        with open(error_log_file, "w") as log:
            log.write(f"Code de sortie : {e.returncode}\n")
            log.write("Erreur standard (stderr) :\n")
            log.write(e.stderr + "\n")

# Fonction principale
for input_dir in input_dirs:
    dataset_name = Path(input_dir).stem
    output_dir = output_base_dir / dataset_name
    output_dir.mkdir(exist_ok=True)
    
    for file in os.listdir(input_dir):
        if file.endswith(".xyz"):
            mol_id = os.path.splitext(file)[0]
            with tempfile.NamedTemporaryFile(delete=False, suffix=".xyz") as temp_xyz:
                format_xyz_file(os.path.join(input_dir, file), temp_xyz.name)
                run_orca(mol_id, temp_xyz.name, output_dir)
                os.unlink(temp_xyz.name)  # Supprimer le fichier temporaire

# Extraction des données ORCA
au2ev = 27.2114
for dataset_dir in output_base_dir.iterdir():
    if dataset_dir.is_dir():
        data = []
        output_csv = dataset_dir / "energies.csv"
        output_pkl = dataset_dir / "energies.pkl"
        
        for file in dataset_dir.glob("Test_*.out"):
            mol_id = file.stem.split("_")[1]
            with open(file, "r") as f:
                text_content = f.readlines()
            
            orbital_energies = []
            homo, lumo = None, None
            
            for i, line in enumerate(text_content):
                if "ORBITAL ENERGIES" in line:
                    start_index = i + 3
                    break
            else:
                print(f"⚠️ ORBITAL ENERGIES non trouvé pour {mol_id}. Fichier ignoré.")
                continue

            for line in text_content[start_index:]:
                if line.strip() == "" or "*" in line:
                    break
                parts = line.split()
                if len(parts) < 4:
                    continue
                try:
                    occ = float(parts[1])
                    e_ev = float(parts[3])
                except ValueError:
                    continue
                orbital_energies.append((occ, e_ev))

            for occ, e_ev in orbital_energies:
                if occ > 0:
                    homo = e_ev
                else:
                    lumo = e_ev
                    break

            gap = (lumo - homo) if homo is not None and lumo is not None else None
            data.append({"mol_id": mol_id, "HOMO(eV)": homo, "LUMO(eV)": lumo, "GAP (eV)": gap})
        
        with open(output_csv, "w", newline="") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=["mol_id", "HOMO (eV)", "LUMO (eV)", "HOMO-LUMO GAP (eV)"])
            writer.writeheader()
            writer.writerows(data)

        with open(output_pkl, "wb") as pklfile:
            pickle.dump(data, pklfile)

        print(f"✅ Résultats enregistrés dans {output_csv} et {output_pkl}.")


In [5]:

# Constantes de conversion
au2ev = 27.2114  # Conversion de Hartree en eV

# Dossier contenant les fichiers de sortie ORCA
orca_dir = Path("./path/orca_results")

# Fichiers de sortie CSV et PKL
output_csv = orca_dir / "energies.csv"
output_pkl = orca_dir / "energies.pkl"

# Initialisation des données à stocker
data = []

# Parcourir tous les fichiers de sortie ORCA
for file in orca_dir.glob("Test_*.out"):
    mol_id = file.stem.split("_")[1]  # Extraire l'ID de la molécule
    with open(file, "r") as f:
        text_content = f.readlines()

    orbital_energies = []
    homo = None
    lumo = None

    # Trouver le bloc contenant "ORBITAL ENERGIES"
    for i, line in enumerate(text_content):
        if "ORBITAL ENERGIES" in line:
            start_index = i + 3  # Commencer après l'en-tête
            break
    else:
        print(f"⚠️ ORBITAL ENERGIES non trouvé pour {mol_id}. Fichier ignoré.")
        continue  # Passer au fichier suivant si aucun bloc trouvé

    # Lire les énergies des orbitales
    for line in text_content[start_index:]:
        if line.strip() == "" or "*" in line:  # Arrêter si fin de section
            break
        parts = line.split()
        if len(parts) < 4:
            continue  # Ignorer les lignes mal formatées

        try:
            occ = float(parts[1])  # Occupation de l'orbitale
            e_ev = float(parts[3])  # Énergie en eV
        except ValueError:
            continue  # Ignorer les erreurs de conversion

        orbital_energies.append((occ, e_ev))

    # Trouver HOMO (dernière orbitale occupée) et LUMO (première orbitale vide)
    for occ, e_ev in orbital_energies:
        if occ > 0:  # Orbitale occupée
            homo = e_ev
        else:  # Orbitale virtuelle (vide)
            lumo = e_ev
            break  # Arrêter dès qu'on trouve la LUMO

    # Calculer le gap si possible
    gap = (lumo - homo) if homo is not None and lumo is not None else None

    # Stocker les résultats
    data.append({
        "mol_id": mol_id,
        "HOMO_ORCA(eV)": homo,
        "LUMO_ORCA(eV)": lumo,
        "GAP_ORCA(eV)": gap,
    })

# Écrire les résultats dans un fichier CSV
with open(output_csv, "w", newline="") as csvfile:
    fieldnames = ["mol_id", "HOMO_ORCA(eV)", "LUMO_ORCA(eV)", "GAP_ORCA(eV)"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for entry in data:
        writer.writerow(entry)

# Écrire les résultats dans un fichier PKL
with open(output_pkl, "wb") as pklfile:
    pickle.dump(data, pklfile)

print(f"✅ Les données ont été enregistrées dans {output_csv} et {output_pkl}.")


✅ Les données ont été enregistrées dans path/orca_results/energies.csv et path/orca_results/energies.pkl.
